TextGen with Markov chains and weirdly compatible datasets

## часть 1: марковские цепи своими руками

**подготовительный этап**

сначала нам нужно задать некоторый текст, на основе которого мы будем генерировать новые предложения. Создадим документ, в который добавим текст

In [2]:
! touch "text.txt" # это команда для linux-подобных систем, для Win попробуйте ni (в PowerShell)

следующей командой добавим текст (например, о приготовлении кофе) в документ, сразу в терминале

In [3]:
! echo "This is a manual brewing process where grounds are soaked in hot water then pressed down to the bottom of the coffee maker by pushing down on a plunger. A French press brews the fullest-flavored cup of coffee, thanks to its stainless steel filter. While a paper filter soaks up the coffee's natural oils (where much of the flavor is held), the press' built-in metal filter allows the oils through. Note: Because a French press does not use a paper filter, there will be a little coffee sediment at the bottom, so leave that last sip in the mug." >> text.txt

In [4]:
! tail -1 text.txt # теперь посмотрим на текст: head покажет начало документа, tail - конец

tail: option used in invalid context -- 1


**основной этап**

In [6]:
# напишем питонскую функцию которая генерирует новые последовательности слов из заданнного текста


def markov_text(input_text):
    """ This func opens a file, reads its contents and 
    creates a dictionary of words in the markov_gen variable
    based on the number of words a user wants to generate.
    """
    import random
    import nltk
    import ssl

    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        pass
    else:
        ssl._create_default_https_context = _create_unverified_https_context


    nltk.download("punkt", quiet=True)
    from nltk import word_tokenize


    data_sample = input_text

    text_data = open(data_sample, 'r').read()
    text_data = ''.join([i for i in text_data]).replace("\\n", "").replace("\n", "")
    text_data = word_tokenize(text_data)
    # we made a list out of words 
    
    index = 1 # задали первый индекс
    markov_gen = {} # пустой словарь, в него положим ключи(слова) и значения (слова, рядом с ними)
    word_count = int(input('select the number of words you want to generate ')) 
    # задали кол-во итераций: сколько слов будет в итоговом тексте

    for character in text_data[index:]: 
        # для каждого в списке слов датасета, начиная со второго слова
        
        key = text_data[index-1] # ключ - это предыдущее слово
        if key in markov_gen:
            markov_gen[key].append(character)
            # если слово уже в ключах, добавляем к нему в лист значений новое
        else:
            markov_gen[key] = [character] # в ином создаем лист для значений
        index += 1

    character1 = random.choice(list(markov_gen.keys()))
    # превращаем ключи в лист, рандомчойсом выбираем начало
    message = character1.capitalize()

    while len(message.split(' ')) < word_count: # пока слов меньше лимита итераций
        character2 = random.choice(markov_gen[character1]) 
        # рандомно выбираем слово из значений, ключ - предыдущее слово
        character1 = character2 
        message += ' ' + character2
        
    return message

теперь можно применить функцию к текстовому документу

In [8]:
markov_text("text.txt")

select the number of words you want to generate 17


"A French press ' built-in metal filter . Note : Because a paper filter , thanks to"

Мы сгенерировали новое предложение длиной в 17 слов, на тему приготовления кофе. *(пока что они выглядят странно, это нормально)*

## часть 2 : попробуем генерировать истории


In [12]:
! pip3 install markovify # установим библиотеку для марковских цепей

ERROR: Invalid requirement: '#'


In [11]:
import markovify 

попробуем [на текстах басен Крылова](https://github.com/sjut/DPO_Materials/blob/master/%D0%94%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5%20%D0%B8%20%D1%81%D1%81%D1%8B%D0%BB%D0%BA%D0%B8/basni.zip):

In [9]:
# сохраним текстовый файл, откроем его

with open("test_basni.txt", encoding = "utf-8") as f:
    data = f.read()

In [14]:
#  print(data) # здесь можно посмотреть на содержимое файла

In [18]:
initial_letters = markovify.Text(data, state_size =1) 
# последний аргумент регулирует размер повторяющихся цепочек

for i in range(4): # задаем количество предложений 
    print(initial_letters.make_sentence(), sep = "\n")

Ударили в землю лбом; «Изрядно, — с высоты: От первых жди от них разборы И поплатился он ослабевал И споры, кому и нежен, и — сказать неложно, Тебя без сил, Валяется в старости и протяжно: Во всем лесу у нас была царь-птица!» Вещуньина с высоты: От первых жди от них разборы И на лад нейдет.
Поверишь ли, в смычки, дерут, а толку нет.
Уж сколько раз твердили миру, Что лесть гнусна, вредна; но только замолчишь, то жду я, не сводит И сели на лад нейдет.
Поверишь ли, в сердце льстец всегда отыщет уголок.


Мы можем задавать и количество слов в предложении, и количество предложений

In [19]:
for i in range(3): # кол-во предложений
    print(initial_letters.make_short_sentence(80), sep = "\n") # кол-во слов в предложении

Ведь вы не стыдись!
Ведь вы не так сладко, чуть дыша: «Голубушка, как сидеть.
Расселись, начали Квартет; Он все-таки на шум их прилететь.


## часть 3: что делать дальше

Попробуйте сгенерировать интересные тексты из материалов, которые интересны Вам: художественная литература, новости, статьи, и  тд. Как правило, чем больше изначального текста, тем больше вероятность, что генерированный текст будет выглядеть естественно

In [1]:
! pip3 install markovify --q

In [20]:
import markovify
import re

In [21]:
with open("Strugackiy.txt", encoding = "utf-8") as f:
    Strugackiy = f.read()    

In [22]:
with open("Huxley.txt", encoding = "utf-8") as f:
    Huxley = f.read()

In [23]:
model_a = markovify.Text(Strugackiy)
model_b = markovify.Text(Huxley)

model_combo = markovify.combine([ model_a, model_b ])

In [726]:
for i in range(3):
    print(model_combo.make_sentence(), sep = "\n")

Сидоров перешагнул через Валькенштейна и подошел к оператору: — Пойдемте.
И опять Кондратьев не стал переспрашивать: он решил, что Бернарду и впрямь, пожалуй, влили спирту в кровезаменитель. — Фосген, хлорпикрин, йодуксусный этил, дифенилцианарсин, слезоточивый газ, иприт.
В техническом аспекте проще простого было бы зарыть или сжечь. — И вы такой же, как все.


In [778]:
for i in range(3):
    print(model_combo.make_short_sentence(90))

И они тянут, тянут, тянут с высадкой… Почему они не могут? — проворчал Белов.
— Товарищ, — сказал диспетчер сквозь зубы.
Он вспомнил, что это очень противно, когда тебе плачут в жилетку.
